# Estudo de Self-Attention

### Gerando Inputs

In [1]:
import torch
import numpy as np

Sentenças Fictícias:

- o mikas ta ali
- eu gosto de batata frita
- existem elefantes

Informações
- O tamanho da maior sentença é 5.
- Temos 3 exemplos de teste.
- Usaremos inteiros para representar cada palavra.
- Usaremos 0 para representar os espaços vazios das sentenças.
- O tamanho do nosso vocabulário é 12 pois temos 12 palavras distintas.


In [2]:
sentence_matrix = torch.LongTensor([
    [1,   2,  3,  4, 0], #       o     mikas   ta      ali
    [5,   6,  7,  8, 9], #      eu     gosto   de   batata   frita
    [10, 11,  0,  0, 0], # existem elefantes

])
qnt_sentences, max_sentence_len = sentence_matrix.shape
print(f"Temos {qnt_sentences} sentenças e a maior sentença tem {max_sentence_len} tokens.")
sentence_matrix.shape

Temos 3 sentenças e a maior sentença tem 5 tokens.


torch.Size([3, 5])

Em geral não usamos representações inteiras para representar palavras, mas sim embedding.

Geraremos embeddings fictícios de tamanho 10 para os 12 tokens disponíveis no vocabulário.

In [3]:
vocab_size = len(sentence_matrix.unique())
embedding_size = 10
embeddings = np.random.random((vocab_size, embedding_size))
# Convertendo para tensor:
embeddings = torch.Tensor(embeddings)
print(f"Temos {max_sentence_len} embeddings de tamanho {embedding_size}.")

Temos 5 embeddings de tamanho 10.


Agora geraremos a matriz de input para os modelos.

- Nela teremos o shape 3, 5, 10 que representam as 3 sentenças com no máximo 5 tokens.
- Cada token é representado por um embedding de tamanho 10.

In [4]:
indexes = sentence_matrix
input_matrix = embeddings[indexes]
print(f"Temos {input_matrix.shape[0]} instancias de no máximo {input_matrix.shape[1]} tokens representados por embeddings de tamanho {input_matrix.shape[2]}.")
input_matrix.shape

Temos 3 instancias de no máximo 5 tokens representados por embeddings de tamanho 10.


torch.Size([3, 5, 10])

### Passando inputs pela camada de RNN:

A atenção é gerada a partir dos dados gerados na camada de RNN, portanto passaremos o input pela camada de RNN:

In [5]:
rnn = torch.nn.RNN(input_size=embedding_size, hidden_size=100, batch_first=True)
output, hidden = rnn(input_matrix)
print(output.shape)
print(hidden.shape)

torch.Size([3, 5, 100])
torch.Size([1, 3, 100])


In [6]:
output[0][4], hidden[0][0]

(tensor([ 9.4811e-02,  1.3441e-02,  9.6388e-02, -8.9834e-02,  1.8093e-01,
          2.6396e-01, -1.8504e-01,  1.5269e-01, -6.6017e-02,  1.7505e-02,
          1.1906e-01,  3.3337e-02,  2.8739e-01, -1.5441e-01, -2.2413e-02,
          1.8803e-01,  1.7794e-01,  2.5703e-01, -1.2328e-01,  1.8018e-01,
          2.6090e-02,  9.8289e-02, -4.7861e-02, -9.9889e-02, -2.8985e-02,
          4.1652e-02, -1.4378e-01,  1.2372e-01, -1.7425e-01, -3.1504e-01,
         -7.0133e-02, -3.0654e-02,  2.6089e-02,  8.3277e-02,  3.3576e-02,
          8.7471e-02,  3.2080e-01,  1.6881e-01, -4.8522e-02, -9.6803e-02,
          2.4317e-04, -2.6249e-01, -4.5399e-02,  7.3564e-02, -6.7889e-02,
         -8.3345e-02,  2.2448e-01, -3.3408e-02,  4.3858e-02,  1.0946e-01,
         -1.0805e-01,  5.6446e-02,  1.7231e-02, -7.0411e-02,  2.8738e-01,
          2.0808e-01,  2.4116e-01,  2.0884e-01, -4.7129e-02, -5.2680e-02,
          1.8771e-02,  1.8045e-01, -3.0577e-01, -2.4539e-01,  1.2513e-01,
          1.9099e-01,  1.4139e-01, -2.